# This kernel contains the solution codes for House Prices: Advanced Regression Techniques competion of www.kaggle.com website

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges us to predict the final price of each home

The visualization for all the variables to derive necessary information has been done in house prices visualization file

In [1]:
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np # fundamental package for scientific computing with Python

# Supress unnecessary warnings so that presentation looks clean
import warnings
warnings.filterwarnings("ignore")

# for plotting
import matplotlib.pyplot as plt

# Print all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

We will  read the train and test datasets using pandas

In [2]:
train = pd.read_csv("train.csv") # reading the train dataset
test = pd.read_csv("test.csv") # reading the test dataset

We will remove some outliers from the train dataset. The outliers that would be removed was decided in the visualization file

In [3]:
print ("Number of rows in train dataset before removing outliers is " + str(train.shape[0]))
# The visualization for the three variables has been done in the House Prices Visualization file. 
# The outliers have been identified from the visualization that we had done
train = train[train.GrLivArea < 4000] # Removing the outliers for Grade living area
train = train[train.TotalBsmtSF < 6000] # Removing the outliers for Total Basement Sq. footage
train = train[train.LotArea < 150000] # Removing the outliers for Total Lot Area
print ("Number of rows in train dataset after removing outliers is " + str(train.shape[0]))

Number of rows in train dataset before removing outliers is 1460
Number of rows in train dataset after removing outliers is 1453


In [4]:
# lets print the head of train to get an idea
train.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


In [5]:
# also print the head of test
test.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal


Lets store the target variable which is SalePrice in a seperate series

In [6]:
target = train["SalePrice"]
# Log transforming the target as per the requirement in the competition for measuring the root mean squared value
target_log = np.log1p(target)

Lets store the feature variables in the train dataframe as the target variable has already been stored in a seperate series

In [7]:
train = train[train.columns.values[:-1]] # Taking the predictors in the train dataframe as train dataset
train = train.drop(['Id'], axis  = 1) # Dropping the Id variable as it would not be of any much help in predicting the target values
test_Ids = test.Id #Storing the test ids in a seperate variable for later prediction
test = test.drop(['Id'], axis  = 1) # Dropping the Id variable as it would not be of any much help for predicting the target values

In [8]:
# joining the train and test variables for preventing multiple operations on train and test seperately
df_original = train.append(test, ignore_index = True)

In [9]:
# let us find the features which have missing values
for col in df_original.columns.values:
    if np.sum(df_original[col].isnull())>0:
        value = np.sum(df_original[col].isnull())
        print(col,value) # printing the column names along with the missing values related to the columns

MSZoning 4
LotFrontage 484
Alley 2714
Utilities 2
Exterior1st 1
Exterior2nd 1
MasVnrType 24
MasVnrArea 23
BsmtQual 81
BsmtCond 82
BsmtExposure 82
BsmtFinType1 79
BsmtFinSF1 1
BsmtFinType2 80
BsmtFinSF2 1
BsmtUnfSF 1
TotalBsmtSF 1
Electrical 1
BsmtFullBath 2
BsmtHalfBath 2
KitchenQual 1
Functional 2
FireplaceQu 1420
GarageType 157
GarageYrBlt 159
GarageFinish 159
GarageCars 1
GarageArea 1
GarageQual 159
GarageCond 159
PoolQC 2904
Fence 2342
MiscFeature 2809
SaleType 1


In [10]:
# As per the descriptions given about the data, below are the variables where NA means the feature is not present in a house
# So we are going to fill the NA values with 'none'
cols = ['Alley','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType',
       'GarageFinish','GarageQual','GarageCond','Fence','MiscFeature']
for col in cols:
    df_original[col] = df_original[col].fillna('none') # filling the na values with none value

In [11]:
# Pool is only present in 10 samples out of 2919 samples so dropping PoolArea and PoolQC features from the dataframe
df_original = df_original.drop(['PoolArea','PoolQC'], axis=1) # dropping the pool features

In [12]:
# Fill the missing value of lot frontage with median
df_original['LotFrontage'] = df_original['LotFrontage'].fillna(df_original['LotFrontage'].median())

In [13]:
# There are several categorical variables where there are missing values denoted as NA
# We will fill those missing values of categorical variables with the most frequent values
cols = ['MSZoning','Utilities','Exterior1st','Exterior2nd','MasVnrType',
        'Electrical','BsmtFullBath','BsmtHalfBath','KitchenQual','Functional','SaleType']
for col in cols:
    df_original[col] = df_original[col].fillna(df_original[col].value_counts().index[0]) 

We are going to map the below variabes as per our understanding, interpretation and also the visualization that we have done earlier in the house prices visualization file

In [14]:
# Feature MSSubClass is not a continuous variable as per the description so decided to change it to a categorical variable
df_original['MSSubClass'] = df_original['MSSubClass'].map({20:'S20',30:'S30',40:'S40',45:'S45',50:'S50',60:'S60',
                                                           70:'S70',75:'S75',80:'S80',85:'S85',90:'S90',120:'S120',
                                                           150:'S150',160:'S160',180:'S180',190:'S190'})

# For feature MSZoning decided to club the Residential Low Density Park(RP) and Residential Medium Density(RM). Refer to
# visualization of this feature in the House Prices Visualization file
df_original['MSZoning'] = df_original['MSZoning'].map({'RL':0,'RM':1,'RP':1,'RH':2,'I':3,'FV':4,'C (all)':5,'A':6}).astype(object)

# For feature LotShape decided to club Irregular lot shapes(IRs) together as per the data description
df_original['LotShape'] = df_original['LotShape'].map({'IR1':0,'IR2':0,'IR3':0,'Reg':1}).astype(object)

# For feature LotConfig decided to club FR2(Frontage on 2 sides of property) and FR3(Frontage on 3 sides of property) together
df_original['LotConfig'] = df_original['LotConfig'].map({'Inside':0,'Corner':1,'CulDSac':2,'FR2':3,'FR3':3}).astype(object)

# Made the house style variable into a continuous variable according to the storeys available and whether it is furnished or not
df_original['HouseStyle'] = df_original['HouseStyle'].map({'1Story':1,'2Story':2,'1.5Fin':1.5,'SLvl':2,
                                                           'SFoyer':2,'1.5Unf':1.25,'2.5Unf':2.25,'2.5Fin':2.5})

# Decided to club a few types of Exterior1st variable based on the visualization in the House Prices Visualization file and also
# very less number of some types available in the dataset
df_original['Exterior1st'] = df_original['Exterior1st'].map({'VinylSd':0,
                                                             'HdBoard':1,'MetalSd':1,'Wd Sdng':1,
                                                             'Plywood':2,
                                                             'CemntBd':3,
                                                             'BrkFace':4,'WdShing':4,'Stucco':4,'AsbShng':4,'Stone':4,
                                                             'BrkComm':4,'AsphShn':4,'CBlock':4,'ImStucc':4}).astype(object)

# Decided to club a few types of Exterior1st variable based on the visualization in the House Prices Visualization file and also
# very less number of some types available in the dataset
df_original['Exterior2nd'] = df_original['Exterior2nd'].map({'VinylSd':0,
                                                             'HdBoard':1,'MetalSd':1,'Wd Sdng':1,
                                                             'Plywood':1,'WdShing':1,'Wd Shng':1,'CmentBd':2,
                                                             'Stucco':3,'BrkFace':3,'AsbShng':3,'ImStucc':3,
                                                             'Brk Cmn':3,'Stone':3,'AsphShn':3,'CBlock':3,'Other':3}).astype(object)

# Decided to club a few types of MasVnr variable based on the description of the data. 
# BrkFace and BrkCmn belongs to brick category
df_original['MasVnrType'] = df_original['MasVnrType'].map({'None':0,'CBlock':1,'BrkFace':2,'BrkCmn':2,'Stone':3}).astype(object)

# Decided to club a few types of Foundation variable because of very less number of some types available in the dataset
df_original['Foundation'] = df_original['Foundation'].map({'PConc':0,'CBlock':1,'BrkTil':2,'Slab':3,'Stone':3,'Wood':3}).astype(object)

# Nothing much fancy with BsmtExposure, BsmtFinType1, BsmtFinType2, GarageType, GarageFinish. Just turned the non numerical ones.
# Could have left the features as they were.
df_original['BsmtExposure'] = df_original['BsmtExposure'].map({'Gd':4,'Av':3,'Mn':2,'No':1,'none':0}).astype(object)
df_original['BsmtFinType1'] = df_original['BsmtFinType1'].map({'GLQ':6,'ALQ':5,'BLQ':3,'Rec':3,'LwQ':2,'Unf':1,'none':0}).astype(object)
df_original['BsmtFinType2'] = df_original['BsmtFinType2'].map({'none':0,'Unf':1,'LwQ':2,'Rec':2,'BLQ':2,'ALQ':3,'GLQ':4}).astype(object)
df_original['GarageType'] = df_original['GarageType'].map({'none':0,'Detchd':1,'CarPort':2,'BuiltIn':3,
                                                           'Basment':4,'Attchd':5,'2Types':6}).astype(object)
df_original['GarageFinish'] = df_original['GarageFinish'].map({'none':0,'Unf':1,'RFn':2,'Fin':3}).astype(object)

# For heating variable tried to club all the types with less number of occurences in the dataset
df_original['Heating'] = df_original['Heating'].map({'GasA':0,'GasW':1,'Grav':1,'Wall':1,'OthW':1,'Floor':1}).astype(object)

# Changed the month variable into a categorical variable
df_original['MoSold'] = df_original['MoSold'].replace({1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr',
                                                       5:'May', 6:'Jun', 7:'Jul', 8:'Aug',
                                                       9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'})

# Clubbed the various types of sale type variables based on the description given about the data
df_original['SaleType'] = df_original['SaleType'].map({'Oth':1,
                                                       'ConLD':2,'ConLI':2,'ConLw':2,'Con':2,
                                                       'COD':3,
                                                       'New':4,
                                                       'WD':5,'CWD':5,'VWD':5}).astype(object)

# Clubbed the various types of sale condition variables based on the description given about the data
df_original['SaleCondition'] = df_original['SaleCondition'].map({'Normal':0,'Partial':1,'Abnorml':2,
                                                                 'Family':0,'Alloca':0,'AdjLand':0}).astype(object)

Changing the Quality and Condition variables into continuous variables as they are categorical variables but ordinal in nature. I have always experienced transforming ordinal into continuous variables always help

In [15]:
df_original['ExterQual'] = df_original['ExterQual'].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1})
df_original['ExterCond'] = df_original['ExterCond'].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1})

df_original['BsmtQual'] = df_original['BsmtQual'].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'none':0})
df_original['BsmtCond'] = df_original['BsmtCond'].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'none':0})


df_original['GarageQual'] = df_original['GarageQual'].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'none':0})
df_original['GarageCond'] = df_original['GarageCond'].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'none':0})

df_original['HeatingQC'] = df_original['HeatingQC'].map({'Ex':5,'TA':3,'Gd':4,'Fa':2,'Po':1})
df_original['KitchenQual'] = df_original['KitchenQual'].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1})
df_original['FireplaceQu'] = df_original['FireplaceQu'].map({'none':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})

1. For the below variables, the idea is that if a certain feature is not present then a related feature would be o. Otherwise the missing value for that related feature was filled with the most frequent value. 

2. Suppose if garage is not present then garage area would be 0. But if a garage is present and somehow the area is missing, the missing value in this case would be filled with the most frequent value

In [16]:
for row in range(0,df_original.shape[0]):
    if np.isnan(df_original['GarageCars'][row]):
        if df_original['GarageType'][row] == 0:
            df_original['GarageCars'][row] = 0
        else:
            df_original['GarageCars'][row] = df_original['GarageCars'].value_counts().index[0]
            
    if np.isnan(df_original['GarageArea'][row]):
        if df_original['GarageType'][row] == 0:
            df_original['GarageArea'][row] = 0
        else:
            df_original['GarageArea'][row] = df_original['GarageArea'].value_counts().index[0]
            
    if np.isnan(df_original['GarageYrBlt'][row]):
        if df_original['GarageType'][row] == 0:
            df_original['GarageYrBlt'][row] = df_original['YrSold'][row]
        else:
            df_original['GarageYrBlt'][row] = df_original['GarageYrBlt'].value_counts().index[0]
            
    if np.isnan(df_original['BsmtFinSF1'][row]):
        if df_original['BsmtQual'][row] == 0:
            df_original['BsmtFinSF1'][row] = 0
        else:
            df_original['BsmtFinSF1'][row] = df_original['BsmtFinSF1'].median()
            
    if np.isnan(df_original['BsmtFinSF2'][row]):
        if df_original['BsmtQual'][row] == 0:
            df_original['BsmtFinSF2'][row] = 0
        else:
            df_original['BsmtFinSF2'][row] = df_original['BsmtFinSF2'].median()
            
    if np.isnan(df_original['BsmtUnfSF'][row]):
        if df_original['BsmtQual'][row] == 0:
            df_original['BsmtUnfSF'][row] = 0
        else:
            df_original['BsmtUnfSF'][row] = df_original['BsmtUnfSF'].median()
            
    if np.isnan(df_original['TotalBsmtSF'][row]):
        df_original['TotalBsmtSF'][row] = df_original['BsmtFinSF1'][row] + df_original['BsmtFinSF2'][row] + df_original['BsmtUnfSF'][row]
        
    if np.isnan(df_original['MasVnrArea'][row]):
        if df_original['MasVnrType'][row] == 0:
            df_original['MasVnrArea'][row] = 0
        else:
            df_original['MasVnrArea'][row] = df_original['MasVnrArea'].median()

Some features are created

In [17]:
df_original['Age'] = df_original['YrSold'] - df_original['YearBuilt']
df_original['Garage_Age'] = df_original['YrSold'] - df_original['GarageYrBlt']
df_original['Remodel_Age'] = df_original['YrSold'] - df_original['YearRemodAdd']
df_original['Porch_Area'] = df_original['OpenPorchSF'] + df_original['ScreenPorch'] + df_original['3SsnPorch'] + df_original['EnclosedPorch']

1. The neighborhood class was divided into three classes Average, Good, Excellent based on the feature's visualization with the house prices done in the House Prices Visualization File.

2. For Alley, Basement, Basement Finish Type 1 and Type 2, Fireplaces, Garage, Fence, Miscellaneous features, another feature was engineered for each of them to denote whether they are present or not

3. Season feature was engineered for which months the price tend to be a bit higher. It was visualized in the House Prices visualization file

4. Newer dwelling was also engineered for relatively newer houses

In [18]:
#neighborhood class
df_original['Neighborhood_class'] = np.zeros((df_original.shape[0],1))
df_original['Neighborhood_class'].loc[df_original['Neighborhood_class'] == 0] = 'Average'
df_original['Neighborhood_class'].loc[df_original['Neighborhood'] == 'Veenker'] = 'Good'
df_original['Neighborhood_class'].loc[df_original['Neighborhood'] == 'NoRidge'] = 'Excellent'
df_original['Neighborhood_class'].loc[df_original['Neighborhood'] == 'Somerst'] = 'Good'
df_original['Neighborhood_class'].loc[df_original['Neighborhood'] == 'NridgHt'] = 'Excellent'
df_original['Neighborhood_class'].loc[df_original['Neighborhood'] == 'Timber'] = 'Good'
df_original['Neighborhood_class'].loc[df_original['Neighborhood'] == 'StoneBr'] = 'Excellent'
df_original['Neighborhood_class'] = df_original['Neighborhood_class'].astype(object)

# presence of certain features
df_original['Alley_present'] = np.zeros((df_original.shape[0],1))
df_original['Alley_present'].loc[df_original['Alley'] != 0] = 1

df_original['Bsmt_present'] = np.zeros((df_original.shape[0],1))
df_original['Bsmt_present'].loc[df_original['BsmtQual'] != 0] = 1

df_original['BsmtFinType1_present'] = np.zeros((df_original.shape[0],1))
df_original['BsmtFinType1_present'].loc[df_original['BsmtFinType1'] != 0] = 1

df_original['BsmtFinType2_present'] = np.zeros((df_original.shape[0],1))
df_original['BsmtFinType2_present'].loc[df_original['BsmtFinType2'] != 0] = 1

df_original['Fireplace_present'] = np.zeros((df_original.shape[0],1))
df_original['Fireplace_present'].loc[df_original['FireplaceQu'] != 0] = 1

df_original['Garage_present'] = np.zeros((df_original.shape[0],1))
df_original['Garage_present'].loc[df_original['GarageQual'] != 0] = 1

df_original['Fence_present'] = np.zeros((df_original.shape[0],1))
df_original['Fence_present'].loc[df_original['Fence'] != 0] = 1

df_original['MiscFeature_present'] = np.zeros((df_original.shape[0],1))
df_original['MiscFeature_present'].loc[df_original['MiscFeature'] != 0] = 1


#season
df_original['Season'] = np.zeros((df_original.shape[0],1))
df_original['Season'] = df_original['MoSold'].map({'Jan':0,'Feb':0,'Mar':0,'Apr':0,'May':0,'Jun':0,'Jul':0,
                                                   'Aug':1,'Sep':1,'Oct':0,'Nov':1,'Dec':1})
df_original['Season'] = df_original['Season'].astype(object)


#newer_dwelling
df_original['Newer_dwelling'] = df_original['MSSubClass'].map({'S20':1,'S30':0,'S40':0,'S45':0,'S50':0,
                                                               'S60':1,'S70':0,'S75':0,'S80':0,'S85':0,
                                                               'S90':0,'S120':1,'S150':0,'S160':0,'S180':0,'S190':0})
df_original['Newer_dwelling'] = df_original['Newer_dwelling'].astype(object)

For all the Quality and the Condition variables, Another variable was engineered for dividing them into good(1) or bad(0) quality

In [19]:
df_original['OverallQual_class'] = np.zeros((df_original.shape[0],1))
df_original['OverallQual_class'].loc[df_original['OverallQual'] > 5] = 1

df_original['OverallCond_class'] = np.zeros((df_original.shape[0],1))
df_original['OverallCond_class'].loc[df_original['OverallCond'] > 5] = 1

df_original['ExterQual_class'] = np.zeros((df_original.shape[0],1))
df_original['ExterQual_class'].loc[df_original['ExterQual'] > 3] = 1

df_original['ExterCond_class'] = np.zeros((df_original.shape[0],1))
df_original['ExterCond_class'].loc[df_original['ExterCond'] > 3] = 1

df_original['BsmtQual_class'] = np.zeros((df_original.shape[0],1))
df_original['BsmtQual_class'].loc[df_original['BsmtQual'] > 3] = 1

df_original['BsmtCond_class'] = np.zeros((df_original.shape[0],1))
df_original['BsmtCond_class'].loc[df_original['BsmtCond'] > 3] = 1

df_original['HeatingQC_class'] = np.zeros((df_original.shape[0],1))
df_original['HeatingQC_class'].loc[df_original['HeatingQC'] > 3] = 1

df_original['KitchenQual_class'] = np.zeros((df_original.shape[0],1))
df_original['KitchenQual_class'].loc[df_original['KitchenQual'] > 3] = 1

df_original['FireplaceQu_class'] = np.zeros((df_original.shape[0],1))
df_original['FireplaceQu_class'].loc[df_original['FireplaceQu'] > 3] = 1

df_original['GarageQual_class'] = np.zeros((df_original.shape[0],1))
df_original['GarageQual_class'].loc[df_original['GarageQual'] > 3] = 1

df_original['GarageCond_class'] = np.zeros((df_original.shape[0],1))
df_original['GarageCond_class'].loc[df_original['GarageCond'] > 3] = 1

1. Modifying the variables related to garage a bit
2. If the garage was built after the house was sold then the presence of garages is misleading. So all the variables related to garage will be set to 0

In [20]:
df_original['GarageType'].loc[df_original['Garage_Age'] < 0] = 0
df_original['GarageFinish'].loc[df_original['Garage_Age'] < 0] = 0
df_original['GarageCars'].loc[df_original['Garage_Age'] < 0] = 0
df_original['GarageArea'].loc[df_original['Garage_Age'] < 0] = 0
df_original['GarageQual'].loc[df_original['Garage_Age'] < 0] = 0
df_original['GarageCond'].loc[df_original['Garage_Age'] < 0] = 0

1. Modifying the garage age and Remodel age a bit depending on whether the garage and remodelling was done after or before the selling of the house. Normally the presence of a garage and remodeling will lead to increase of the price of the house but if they were done after the house was sold then they bear no significance to the price of the house

2. Similarly another variable remodel is introduced to denote whether remodelling was done or not

In [21]:
df_original['GarageBuiltAfter'] = np.zeros((df_original.shape[0],1))
df_original['GarageBuiltAfter'].loc[df_original['YearBuilt'] > df_original['GarageYrBlt']] = 1
df_original['Garage_Age'].loc[df_original['Garage_Age'] < 0] = 0

df_original['Remodel'] = np.zeros((df_original.shape[0],1))
df_original['Remodel'].loc[df_original['YearBuilt'] > df_original['YearRemodAdd']] = 1
df_original['Remodel_Age'].loc[df_original['Remodel_Age'] < 0] = 0
df_original['Remodel_Age'].loc[df_original['YearRemodAdd'] == df_original['YearBuilt']] = 0

Dropping the variables year of built, year of remodelling and year of building garage as all these informations have been captured in engineered variables like remodel and remodel_age

In [22]:
df_original = df_original.drop(['YearBuilt','YearRemodAdd','GarageYrBlt'], axis = 1)

Dividing the dataset into categorical and continuous variables for further operation on them

In [23]:
cont = []
for col in df_original.columns.values:
    if df_original[col].dtype != 'object':
        cont.append(col)
df_original_cont = df_original[cont]
df_original_cat = df_original.drop(cont, axis=1)

In [24]:
print("Number of categorical features : " + str(df_original_cat.shape))
print("Number of continuous features : " + str(df_original_cont.shape))

Number of categorical features : (2912, 37)
Number of continuous features : (2912, 65)


The number of continuous variables is far greater than the number of categorical variables

Creating dummies for categorical variables

In [25]:
df_cat_dummies = pd.get_dummies(df_original_cat)

Dropping the features with very less non zero values as they would not have much effect on any model

In [26]:
cols_zero = []
for col in df_cat_dummies.columns.values:
    col_Notzero = np.count_nonzero(df_cat_dummies[col])
    if col_Notzero < 3:
        cols_zero.append(col)
        
df_cat_dummies = df_cat_dummies.drop(cols_zero, axis=1)

We are going to find out which variables are correlated and we are going to remove some correlated variables from the continuous features. The threshold for correlation has been set at 0.8. The indexes of the columns to be removed will be stored in the j_columns

In [27]:
# Correlation tells relation between two attributes.
# Correlation requires continous data. Hence, ignore categorical data
# Calculates pearson co-efficient for all combinations
df_cont_corr = df_original_cont.corr()
# Set the threshold to select only highly correlated attributes
threshold = 0.8
# List of pairs along with correlation above threshold
corr_list = []
# J_columns would contain the indexes of the variables that we are planning to drop
j_columns = []
#Search for the highly correlated pairs
for i in range(0,df_original_cont.shape[1]):
    if i not in np.ravel(j_columns):
        for j in range(i+1,df_original_cont.shape[1]):
            if (df_cont_corr.iloc[i,j] >= threshold and df_cont_corr.iloc[i,j] < 1) or (df_cont_corr.iloc[i,j] < 0 and df_cont_corr.iloc[i,j] <= -threshold):
                corr_list.append([df_cont_corr.iloc[i,j],i,j]) #store correlation and columns index
                j_columns.append([j])

In [28]:
j_columns = np.ravel(j_columns)
j_columns # indexes of columns to drop

array([16, 53, 54, 55, 45, 46, 47, 58, 26, 59, 28, 48, 30, 32, 49])

Lets print the correlation values for the correlated columns whose correlation is greater than 0.8.

In [29]:
for v,i,j in corr_list:
    print ("%s and %s = %.2f" % (i,j,v))

2 and 16 = 0.81
4 and 53 = 0.81
6 and 54 = 0.93
7 and 55 = 0.88
9 and 45 = 0.84
9 and 46 = 0.85
9 and 47 = 0.84
14 and 58 = 0.91
18 and 26 = 0.81
25 and 59 = 0.90
27 and 28 = 0.86
27 and 48 = 0.90
29 and 30 = 0.89
31 and 32 = 0.95
31 and 49 = 0.93


Lets store the names of the features to drop in the list features_to_drop

In [30]:
features_to_drop = []
for i in j_columns:
    features_to_drop.append(df_original_cont.columns[i])

Lets display the names of the features that we are going to drop

In [31]:
features_to_drop

['2ndFlrSF',
 'OverallCond_class',
 'ExterQual_class',
 'ExterCond_class',
 'Bsmt_present',
 'BsmtFinType1_present',
 'BsmtFinType2_present',
 'HeatingQC_class',
 'TotRmsAbvGrd',
 'KitchenQual_class',
 'FireplaceQu',
 'Fireplace_present',
 'GarageArea',
 'GarageCond',
 'Garage_present']

We are going to drop the features which are correlated and they are in the array features_to_drop

In [32]:
df_original_cont = df_original_cont.drop(features_to_drop, axis = 1)

Lets join the continuous and categorical variables into a new dataframe

In [33]:
df_new = df_original_cont.join(df_cat_dummies)
df_new.shape

(2912, 239)

In [34]:
# Lets store the continuous and categorical columns values in a list. We will need them later on
continuous_columns = df_original_cont.columns.values
categorical_columns = df_cat_dummies.columns.values

Lets see if all the missing values in the dataframe has been filled

In [35]:
df_new.isnull().any().any()

False

Lets create the train and target values to fit the predictive algorithm

In [36]:
X_train = df_new.iloc[:train.shape[0],:]
y_train = target_log

Since it is a regression algorithm we are going to use regression algorithms:-  
1. Ridge regression - It is an extension of linear regression. Along with finding the best fit line for regression, it also takes care of overfitting problem. It regularizes the square of the coefficients.
2. Lasso regression - It is similar to Lasso regression. It regularizes the absolute value of the coefficients.
3. Elasticnet regression - It is again an extension of linear regression. It combines the regularization capabilities of Lasso and ridge regression  
The metric that we are going to use for this particular problem is root mean squared error. For more information on root mean squared error, please visit this link https://en.wikipedia.org/wiki/Root-mean-square_deviation

In [37]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
scorer = make_scorer(mean_squared_error, False)

In [38]:
X_train = df_new.iloc[:train.shape[0],:]
y_train = target_log

<b>alpha</b> is the coefficient that is multiplied to square or absolute value of coefficients and this term is added to the cost function

Finding the most optimal value of alpha for Ridge Regression

In [39]:
param = {'alpha':[0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 1, 10, 50]}
model = Ridge()
clf = GridSearchCV(model, param, scoring='neg_mean_squared_error', refit='True', n_jobs=1, cv=5)
clf.fit(X_train, y_train)
print ("The best parameter is " + str(clf.best_params_)) 
print ("Root mean squared error is " + str(np.sqrt(abs(clf.best_score_))))

The best parameter is {'alpha': 10}
Root mean squared error is 0.113956734799


Finding the most optimal value of alpha for Lasso Regression

In [40]:
param = {'alpha':[0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 1, 10, 50]}
model = Lasso()
clf = GridSearchCV(model, param, scoring='neg_mean_squared_error', refit='True', n_jobs=1, cv=5)
clf.fit(X_train, y_train)
print ("The best parameter is " + str(clf.best_params_)) 
print ("Root mean squared error is " + str(np.sqrt(abs(clf.best_score_))))

The best parameter is {'alpha': 0.001}
Root mean squared error is 0.112804542528


Finding the most optimal value of alpha for Elastic Net Regression

In [41]:
from sklearn.linear_model import ElasticNet

In [42]:
# For elastic net regerssion
# if a * L1 + b * L2 is the regularizarin parameter in the cost fucntion where L1 is Lasso coefficients and L2 is Ridge coefficients
# then alpha = a + b and l1_ratio = a / (a + b) in elastic net regression

param = {'alpha':[0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 1, 10, 50],'l1_ratio':[0.2, 0.5, 0.8]}
model = ElasticNet()
clf = GridSearchCV(model, param, scoring='neg_mean_squared_error', refit='True', n_jobs=1, cv=5)
clf.fit(X_train, y_train)
print ("The best parameter is " + str(clf.best_params_)) 
print ("Root mean squared error is " + str(np.sqrt(abs(clf.best_score_))))

The best parameter is {'alpha': 0.001, 'l1_ratio': 0.5}
Root mean squared error is 0.111410838667


In [43]:
X_train = df_new.iloc[0:train.shape[0],:]
y_train = target_log
X_test = df_new.iloc[train.shape[0]:,:]

#### Lets see if standardizing the continuous variables reduces the error

In [44]:
from sklearn.preprocessing import StandardScaler

Standard Scaler standardizes features by removing the mean and scaling to unit variance. We perform standardization of the features by performing a fit_transform function

In [45]:
X_train_cat = X_train[categorical_columns]
X_test_cat = X_test[categorical_columns]

X_train_cont_scaler = X_train[continuous_columns]
X_test_cont_scaler = X_test[continuous_columns]
for col in X_train_cont_scaler.columns.values:
    scaler = StandardScaler().fit(X_train_cont_scaler[col])
    X_train_cont_scaler[col] = scaler.transform(X_train_cont_scaler[col].reshape(1,-1))[0]
    X_test_cont_scaler[col] = scaler.transform(X_test_cont_scaler[col].reshape(1,-1))[0]

C:\Users\Abhishek.penumbra\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\Abhishek.penumbra\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\Abhishek.penumbra\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(

In [46]:
# Lets join the continuous and categorical variables together to get the whole dataset again
X_train_scaler = X_train_cont_scaler.join(X_train_cat)
X_test_scaler = X_test_cont_scaler.join(X_test_cat)

Finding the most optimal value of alpha for <b>Ridge Regression</b> after normalizing the continuous variables

In [47]:
param = {'alpha':[0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 1, 10, 50]}
model = Ridge()
clf = GridSearchCV(model, param, scoring='neg_mean_squared_error', refit='True', n_jobs=1, cv=5)
clf.fit(X_train_scaler, y_train)
print ("The best parameter is " + str(clf.best_params_)) 
print ("Root mean squared error is " + str(np.sqrt(abs(clf.best_score_))))

The best parameter is {'alpha': 10}
Root mean squared error is 0.114010796491


Finding the most optimal value of alpha for Lasso Regression after normalizing the continuous variables

In [48]:
param = {'alpha':[0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 1, 10, 50]}
model = Lasso()
clf = GridSearchCV(model, param, scoring='neg_mean_squared_error', refit='True', n_jobs=1, cv=5)
clf.fit(X_train_scaler, y_train)
print ("The best parameter is " + str(clf.best_params_)) 
print ("Root mean squared error is " + str(np.sqrt(abs(clf.best_score_))))

The best parameter is {'alpha': 0.001}
Root mean squared error is 0.111810562195


In [49]:
param = {'alpha':[0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 1, 10, 50],'l1_ratio':[0.2, 0.5, 0.8]}
model = ElasticNet()
clf = GridSearchCV(model, param, scoring='neg_mean_squared_error', refit='True', n_jobs=1, cv=5)
clf.fit(X_train_scaler, y_train)
print ("The best parameter is " + str(clf.best_params_)) 
print ("Root mean squared error is " + str(np.sqrt(abs(clf.best_score_))))

The best parameter is {'alpha': 0.001, 'l1_ratio': 0.5}
Root mean squared error is 0.11105845109


After standardizing the continuous features, we could improve the scores of Lasso regression and Elastic net regression

Creating the final submission and we are going to take elastic net (It gave us our best cv score) as our model and the parameters have been taken from the cross validation operation done earlier

In [50]:
favoriteClf_EN = ElasticNet(alpha = 0.001, l1_ratio=0.5, max_iter=10000)
favoriteClf_EN.fit(X_train_scaler,y_train)

ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=10000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

Lets transform the predictions back to the real values by applying the exponential function

In [51]:
Predict_EN = np.expm1(favoriteClf_EN.predict(X_test_scaler))

Lets prepare the submission file for Elasticnet predictions

In [52]:
submission = pd.DataFrame(Predict_EN, columns=["SalePrice"])
submission.insert(0, 'id', test_Ids)
submission.reset_index()
submission.to_csv('HousePricesReg.csv', index = False) #0.11895 leaderboard score

We are going to try and see if random forest and xgboost does any better than the regression methods

Lets try out random forest first. For tree based modeling techniques we donot have to use thestandardized dataset

In [53]:
from sklearn.ensemble import RandomForestRegressor

Finding the optimal value of trees for random forest

In [70]:
param = {'n_estimators' : [500,1000,2000,3000]} # the number of treesof the random forests
model = RandomForestRegressor(random_state=0) # fixing the random state for reproducibility
clf = GridSearchCV(model, param, scoring='neg_mean_squared_error', refit='True', n_jobs=1, cv=5)
clf.fit(X_train, y_train)
print ("The best parameter is " + str(clf.best_params_)) 
print ("Root mean squared error is " + str(np.sqrt(abs(clf.best_score_))))

The best parameter is {'n_estimators': 2000}
Root mean squared error is 0.135889943061


Using random forest we are going to predict the house prices and store it in another submission file

In [71]:
Predict_RFR = np.expm1(clf.predict(X_test))

Lets prepare the submission file for Random Forest Regression predictions

In [72]:
submission = pd.DataFrame(Predict_RFR, columns=["SalePrice"])
submission.insert(0, 'id', test_Ids)
submission.reset_index()
submission.to_csv('HousePricesRFR.csv', index = False) # 0.14548

Lets try xgboost now

In [73]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score

We are going to select number of trees as 2500, number of features as 0.8, maximum depth as 4 and subsample as 0.8 and see if xgboost can give us a mindblowing performance. Since Xgboost is relatively computationally expensive when number of trees is in the range of 5000s, so we just selected parameter values based on our discretion. We are not going to run gridsearch on it.

In [86]:
model_Xgb = xgb.XGBRegressor(colsample_bytree=0.8, gamma=0.0,learning_rate=0.01,max_depth=4,
                             min_child_weight=4,n_estimators=2500,reg_alpha=0.9,reg_lambda=0.6,
                             subsample=0.8,seed=42,silent=0)
Score_Xgb_entry = np.sqrt(-cross_val_score(model_Xgb, X_train, y_train, scoring="neg_mean_squared_error", cv = 5)).mean()
print(Score_Xgb_entry)

0.114374189676


Using xgboost we are going to predict the house prices and store it in another submission file

In [75]:
model_Xgb.fit(X_train,y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.0, learning_rate=0.01, max_delta_step=0, max_depth=4,
       min_child_weight=4, missing=None, n_estimators=2500, nthread=-1,
       objective='reg:linear', reg_alpha=0.9, reg_lambda=0.6,
       scale_pos_weight=1, seed=42, silent=0, subsample=0.8)

In [76]:
Predict_xgb = np.expm1(model_Xgb.predict(X_test))

In [77]:
submission = pd.DataFrame(Predict_xgb, columns=["SalePrice"])
submission.insert(0, 'id', test_Ids)
submission.reset_index()
submission.to_csv('HousePricesXGB.csv', index = False) #0.12310

In [78]:
HousePricesRFR = pd.read_csv("HousePricesRFR.csv")
HousePricesXGB = pd.read_csv("HousePricesXGB.csv")
HousePricesReg = pd.read_csv("HousePricesReg.csv")

Lets use some learning model to see if we can get a better score. For deep learning we have to use the standardized dataset

In [79]:
import keras # importing keras
from keras.models import Sequential # importing the type of model that we are gong to use
from keras.layers import Dense, Activation # importing the densely connected neural netwok layer, activation is for introducing non liearity to the output
from keras import optimizers # importing the optimizers to modify the weights later on
from keras.wrappers.scikit_learn import KerasRegressor # importing the regressor of keras
from sklearn.model_selection import KFold, cross_val_score # to find the local cv score

In [80]:
def baseline_model(): # defining the architecture of the NN 
    model = Sequential()
    model.add(Dense(200, input_shape=(X_train.shape[1],), init = 'truncated_normal', activation = 'relu')) # first hidden layer
    model.add(Dense(100, init = 'normal', activation = 'relu')) # second hidden layer
    model.add(Dense(50, init = 'normal', activation = 'sigmoid')) # third hidden layer
    model.add(Dense(1)) # output layer
    op = optimizers.sgd(lr=0.01) # optimizer 
    model.compile(loss='mean_squared_error', optimizer=op) # configuration of the learning process
    
    return model

In [81]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# fix the number of epochs and the batch size
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=64, verbose=0)

In [82]:
kfold = KFold(n_splits=5, random_state=seed)
results = cross_val_score(estimator, X_train_scaler.values, y_train.values, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (np.sqrt(results).mean(), np.sqrt(results).std())) # printing the local cv score for kerass

Results: 0.16 (0.01) MSE


The local cv score for <b>keras</b> is quite high compared to the linear regression methods. One reason can be that it is not getting enough data to learn properly

For the final submission we are going to use an ensemble model which will have a weightage of 0.05 for random forest, 0.3 for xgboost and 0.65 for Elastic net regression. The weights are completely based on our own discretion

In [87]:
HousePrices = 0.1 * HousePricesRFR + 0.2 * HousePricesXGB + 0.7 * HousePricesReg # Producing an ensemble model
# This model is giving a public leaderboard score of 0.11861

In [88]:
submission = pd.DataFrame(HousePrices, columns=["SalePrice"])
submission.insert(0, 'id', test_Ids)
submission.reset_index()
submission.to_csv('HousePricesSubmission.csv', index = False)